In [28]:
import nltk

from nltk.corpus import twitter_samples
from nltk.tokenize import word_tokenize,sent_tokenize

import string
import re
 
from nltk.corpus import stopwords 
stopwords_english = stopwords.words('english')
 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

print (twitter_samples.fileids())


pos_tweets = twitter_samples.strings('positive_tweets.json')

neg_tweets = twitter_samples.strings('negative_tweets.json')

 
all_tweets = twitter_samples.strings('tweets.20150430-223406.json')
# Happy Emotions
emotions_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emotions
emotions_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

emotions = emotions_happy.union(emotions_sad)


def clean_tweets(tweet):
    
    # remove  retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    #remove new lines
    tweet = re.sub('\n',' ',tweet)
     
    # remove urls
    tweet = re.sub(r'https?:\/\/.*[\r\n]*|www.*?\s', '', tweet)
    
    # remove html characters
    tweet = re.sub(r'#|%|&|—', '', tweet)
 
    #tweet_tokens = word_tokenize(tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
 
    new_tweets = []
    
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stop words
                word not in string.punctuation and word not in emotions ): # remove punctuation and emotions
            #tweets lemmatization
            #new_tweets.append(lemmatizer.lemmatize(word.lower(),pos="v"))
            new_tweets.append(word)

 
    return new_tweets



['negative_tweets.json', 'positive_tweets.json', 'tweets.20150430-223406.json']


In [35]:
print (pos_tweets[19])
print ((clean_tweets(pos_tweets[19])))



Hello I need to know something can u fm me on Twitter?? — sure thing :) dm me x http://t.co/W6Dy130BV7
['hello', 'need', 'know', 'something', 'u', 'fm', 'twitter', 'sure', 'thing', 'dm', 'x']


In [5]:
print (nltk.pos_tag(clean_tweets(pos_tweets[15])))


[('lay', 'NN'), ('greet', 'NN'), ('card', 'NN'), ('range', 'NN'), ('print', 'NN'), ('today', 'NN'), ('love', 'VBP'), ('job', 'NN')]


In [30]:
clean_tweets(pos_tweets[6])

['like',
 'keep',
 'lovely',
 'customers',
 'waiting',
 'long',
 'hope',
 'enjoy',
 'happy',
 'friday',
 'lwwf']

In [20]:
pos_tweets[6]

"We don't like to keep our lovely customers waiting for long! We hope you enjoy! Happy Friday! - LWWF :) https://t.co/smyYriipxI"

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(binary=True)
vec.fit(clean_tweets(pos_tweets[6]))
print([w for w in vec.vocabulary_.keys()])

['like', 'keep', 'lovely', 'customers', 'waiting', 'long', 'hope', 'enjoy', 'happy', 'friday', 'lwwf']


In [33]:
import pandas as pd
pd.DataFrame(vec.transform(clean_tweets(pos_tweets[6])).toarray(), columns=vec.vocabulary_.keys())

,like,keep,lovely,customers,waiting,long,hope,enjoy,happy,friday,lwwf
0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,1,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0,0,0,0
8,0,0,0,1,0,0,0,0,0,0,0
9,0,0,1,0,0,0,0,0,0,0,0


In [34]:
def bag_of_words(tweet):
    words = clean_tweets(tweet)
    words_dictionary = dict([word, 1] for word in words)    
    return words_dictionary
pos_tweets_set = []
for tweet in pos_tweets:
    pos_tweets_set.append((bag_of_words(tweet), 'positive tweet'))    
 
neg_tweets_set = []
for tweet in neg_tweets:
    
    neg_tweets_set.append((bag_of_words(tweet), 'negative tweet'))
print (len(pos_tweets_set), len(neg_tweets_set))    

5000 5000


In [4]:
from random import shuffle 
shuffle(pos_tweets_set)
shuffle(neg_tweets_set)
 
test_set = pos_tweets_set[:1000] + neg_tweets_set[:1000]
train_set = pos_tweets_set[1000:] + neg_tweets_set[1000:]
 
print(len(test_set),  len(train_set))

2000 8000


In [5]:
from nltk import classify
from nltk import NaiveBayesClassifier
 
classifier = NaiveBayesClassifier.train(train_set)
 
accuracy = classify.accuracy(classifier, test_set)
print(accuracy)
 


0.776


In [6]:
new_tweet = "the filme was amazing.it was good"
tweet_set = bag_of_words(new_tweet)
print (classifier.classify(tweet_set))

positive tweet


In [7]:
new_tweet = "the filme was boring.it was bad"
tweet_set = bag_of_words(new_tweet)
print (classifier.classify(tweet_set))

negative tweet


In [23]:
from collections import defaultdict
 
actual_set = defaultdict(set)
predicted_set = defaultdict(set)
 
actual_set_cm = []
predicted_set_cm = []
 
for i, (feature, actual_label) in enumerate(test_set):
    actual_set[actual_label].add(index)
    actual_set_cm.append(actual_label)
 
    predicted_label = classifier.classify(feature)
 
    predicted_set[predicted_label].add(index)
    predicted_set_cm.append(predicted_label)
    
from nltk.metrics import ConfusionMatrix
cm = ConfusionMatrix(actual_set_cm, predicted_set_cm)
print (cm)

               |   n   p |
               |   e   o |
               |   g   s |
               |   a   i |
               |   t   t |
               |   i   i |
               |   v   v |
               |   e   e |
               |         |
               |   t   t |
               |   w   w |
               |   e   e |
               |   e   e |
               |   t   t |
---------------+---------+
negative tweet |<758>242 |
positive tweet | 206<794>|
---------------+---------+
(row = reference; col = test)



In [22]:
print (cm.pretty_format(sort_by_count=True, show_percents=True))


               |      n      p |
               |      e      o |
               |      g      s |
               |      a      i |
               |      t      t |
               |      i      i |
               |      v      v |
               |      e      e |
               |               |
               |      t      t |
               |      w      w |
               |      e      e |
               |      e      e |
               |      t      t |
---------------+---------------+
negative tweet | <37.9%> 12.1% |
positive tweet |  10.3% <39.7%>|
---------------+---------------+
(row = reference; col = test)

